# Database Consistency Checks

This notebook checks the top-level `tiles-SPECPROD.(fits|csv)` and `exposures-SPECPROD.(fits|csv)` files for self-consistency in prepartion for loading these files into a database.

## Imports

In [1]:
%matplotlib inline
import os
import glob
import json
from pytz import utc
import numpy as np
import matplotlib.pyplot as plt
import fitsio

from astropy.io import fits
from astropy.table import Table, Column, MaskedColumn, join
from astropy.time import Time

from desiutil.iers import freeze_iers
from desiutil.log import get_logger, DEBUG, INFO

from desispec.io.meta import faflavor2program
from desispec.io.util import checkgzip
import desispec.database.redshift as dsr
from desispec.database.util import targetphotid, surveyid

from desitarget.targets import decode_targetid
freeze_iers()

INFO:iers.py:82:freeze_iers: Freezing IERS table used by astropy time, coordinates.


## Important Setup

In [2]:
specprod = os.environ['SPECPROD'] = 'fuji'
overwrite = True

In [3]:
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'public', 'edr', 'spectro', 'redux')

## Test loading targeting data

In [4]:
release = 'edr'
targetphot_version = 'perlmutter-main'
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'spectro', 'redux')

In [5]:
# targetphot_file = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'vac', 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'observed-targets', f"targetphot-{os.environ['SPECPROD']}.fits")
# targetphot_file = os.path.join(os.environ['SCRATCH'], f"targetphot-{os.environ['SPECPROD']}.fits")
targetphot_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'observed-targets', f"targetphot-{os.environ['SPECPROD']}.fits")
targetphot = Table.read(targetphot_file, hdu=1)

In [6]:
# targetphot_potential_file = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'vac', 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'potential-targets', f"targetphot-potential-{os.environ['SPECPROD']}.fits")
# targetphot_potential_file = os.path.join(os.environ['SCRATCH'], f"targetphot-potential-{os.environ['SPECPROD']}.fits")
targetphot_potential_file = os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'potential-targets', f"targetphot-potential-{os.environ['SPECPROD']}.fits")
targetphot_potential = Table.read(targetphot_potential_file, hdu=1)

In [ ]:
tractorphot_files = glob.glob(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'observed-targets', 'tractorphot', '*.fits'))
tractorphot_potential_files = glob.glob(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'potential-targets', 'tractorphot', '*.fits'))

In [ ]:
tractorphot_ids = Table(np.hstack([fitsio.read(f, columns=['LS_ID', 'TARGETID'], ext='TRACTORPHOT', upper=True) for f in tractorphot_files]))
tractorphot_potential_ids = Table(np.hstack([fitsio.read(f, columns=['LS_ID', 'TARGETID'], ext='TRACTORPHOT', upper=True) for f in tractorphot_potential_files]))

In [ ]:
all_targetids = np.unique(np.hstack((tractorphot_ids['TARGETID'].data, tractorphot_potential_ids['TARGETID'])))

In [7]:
targetphot_surveyid = (np.array([surveyid(ss) for ss in targetphot['SURVEY']], dtype=np.int64) << 32 | targetphot['TILEID'].astype(np.int64))
targetphot_potential_surveyid = (np.array([surveyid(ss) for ss in targetphot_potential['SURVEY']], dtype=np.int64) << 32 | targetphot_potential['TILEID'].astype(np.int64))

In [9]:
targetphot_uniqueid = [int(targetphot_surveyid[k]) << 64 | int(targetphot['TARGETID'][k]) for k in range(len(targetphot_surveyid))]
targetphot_potential_uniqueid = [int(targetphot_potential_surveyid[k]) << 64 | int(targetphot_potential['TARGETID'][k]) for k in range(len(targetphot_potential_surveyid))]

In [13]:
targetphot_uniqueid_index = dict(zip(targetphot_uniqueid, range(len(targetphot))))
targetphot_potential_uniqueid_index = dict(zip(targetphot_potential_uniqueid, range(len(targetphot_potential))))

In [14]:
good_rows = np.ones((len(targetphot_potential),), dtype=np.bool)
for key in targetphot_uniqueid_index:
    good_rows[targetphot_potential_uniqueid_index[key]] = False

/tmp/ipykernel_3477/4228586391.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  good_rows = np.ones((len(targetphot_potential),), dtype=np.bool)


In [26]:
# del targetphot['ID']
# del targetphot_potential['ID']
targetphot.add_column(np.array([targetphot_surveyid, targetphot['TARGETID'].data]).T, name='ID', index=0)
targetphot_potential.add_column(np.array([targetphot_potential_surveyid, targetphot_potential['TARGETID'].data]).T, name='ID', index=0)

In [9]:
j = join(targetphot_potential['SURVEY', 'TILEID', 'TARGETID', 'RELEASE'], targetphot['SURVEY', 'TILEID', 'TARGETID', 'RELEASE'], join_type='left', keys=('SURVEY', 'TILEID', 'TARGETID'), table_names=['POTENTIAL', 'OBSERVED'])

In [12]:
w = j['RELEASE_OBSERVED'].mask

In [29]:
ww = ~j['RELEASE_OBSERVED'].mask
ww

array([False,  True, False, ...,  True,  True, False])

In [19]:
load_surveyid = (np.array([surveyid(s) for s in j['SURVEY'][w]], dtype=np.int64) << 32 | j['TILEID'][w].data.astype(np.int64))
load_id = np.array([load_surveyid, j['TARGETID'][w].data]).T
load_id.shape

(13673720, 2)

In [30]:
omit_surveyid = (np.array([surveyid(s) for s in j['SURVEY'][ww]], dtype=np.int64) << 32 | j['TILEID'][ww].data.astype(np.int64))
omit_id = np.array([omit_surveyid, j['TARGETID'][ww].data]).T
omit_id.shape

(2786841, 2)

In [35]:
targetphot_potential['ID'][0] == omit_id[0,:]

array([ True, False])

In [36]:
good_rows = np.ones((len(targetphot_potential),), dtype=np.bool)
for k in range(omit_id.shape[0]):
    good_rows[(targetphot_potential['ID'] == omit_id[k,:]).all()] = False

/tmp/ipykernel_87666/4137259944.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  good_rows = np.ones((len(targetphot_potential),), dtype=np.bool)


### For a given `TARGETID`, which columns are always the same?

In [ ]:
targetid = targetphot['TARGETID'].data
targetid_potential = targetphot_potential['TARGETID'].data

In [ ]:
unique_targetid, unique_indexes, targetid_indexes, targetid_counts = np.unique(targetid, return_index=True, return_inverse=True, return_counts=True)
assert (unique_targetid[targetid_indexes] == targetid).all()
unique_targetid_potential, unique_indexes_potential, targetid_indexes_potential, targetid_counts_potential = np.unique(targetid_potential, return_index=True, return_inverse=True, return_counts=True)
assert (unique_targetid_potential[targetid_indexes_potential] == targetid_potential).all()

In [ ]:
np.unique(targetid_counts), np.unique(targetid_counts_potential)

In [ ]:
duplicate_targetid = np.nonzero(targetid_counts > 1)[0]
duplicate_targetid_potential = np.nonzero(targetid_counts_potential > 1)[0]

In [ ]:
duplicate_targetid_not_in_tractorphot = set(unique_targetid[duplicate_targetid].tolist()) - set(tractorphot_ids['TARGETID'].tolist())
duplicate_targetid_potential_not_in_tractorphot = set(unique_targetid_potential[duplicate_targetid_potential].tolist()) - set(tractorphot_potential_ids['TARGETID'].tolist())

In [ ]:
len(duplicate_targetid_not_in_tractorphot), len(duplicate_targetid_potential_not_in_tractorphot)

In [ ]:
# Pre-find rows with duplicate TARGETID.
targetphot_cache = os.path.join(os.environ['SCRATCH'], os.path.splitext(os.path.basename(targetphot_file))[0] + '.json')
targetphot_potential_cache = os.path.join(os.environ['SCRATCH'], os.path.splitext(os.path.basename(targetphot_potential_file))[0] + '.json')
if os.path.exists(targetphot_cache):
    print("Loading from cache")
    targetid_to_duplicates = dict()
    with open(targetphot_cache) as fp:
        cache_data = json.load(fp)
        for t in cache_data:
            targetid_to_duplicates[np.int64(t)] = np.array(cache_data[t])
else:
    print("Creating...")
    targetid_to_duplicates = dict()
    i = np.arange(len(targetid), dtype=np.int64)
    c = 0
    for t in duplicate_targetid_not_in_tractorphot:
        targetid_to_duplicates[t] = i[targetid == t].tolist()
        c += 1
        if c % 100 == 0:
            print(c)
    with open(targetphot_cache, 'w') as fp:
        json.dump(targetid_to_duplicates, fp)
if os.path.exists(targetphot_potential_cache):
    print("Loading from cache")
    targetid_to_duplicates_potential = dict()
    with open(targetphot_potential_cache) as fp:
        cache_data = json.load(fp)
        for t in cache_data:
            targetid_to_duplicates_potential[np.int64(t)] = np.array(cache_data[t])
else:
    print("Creating...")
    targetid_to_duplicates_potential = dict()
    i = np.arange(len(targetid_potential), dtype=np.int64)
    c = 0
    for t in duplicate_targetid_potential_not_in_tractorphot:
        targetid_to_duplicates_potential[t] = i[targetid_potential == t].tolist()
        c += 1
        if c % 1000 == 0:
            print(c)
    with open(targetphot_potential_cache, 'w') as fp:
        json.dump(targetid_to_duplicates_potential, fp)

### Examine the columns that differ when TARGETID is the same.

In [ ]:
anomalous_targetid = {'observed': dict(), 'potential': dict()}
for key in anomalous_targetid:
    if key == 'observed':
        dup = targetid_to_duplicates
    else:
        dup = targetid_to_duplicates_potential
    for t in dup:
        if key == 'observed':
            rows = targetphot[dup[t]]
        else:
            rows = targetphot_potential[dup[t]]
        for c in ('RELEASE', 'BRICKID', 'BRICKNAME', 'BRICK_OBJID', 'MORPHTYPE',
                  'RA', 'RA_IVAR', 'DEC', 'DEC_IVAR',
                  'DCHISQ',
                  'EBV',
                  'FLUX_G', 'FLUX_R', 'FLUX_Z',
                  'FLUX_IVAR_G', 'FLUX_IVAR_R', 'FLUX_IVAR_Z',
                  'MW_TRANSMISSION_G', 'MW_TRANSMISSION_R', 'MW_TRANSMISSION_Z',
                  'FRACFLUX_G', 'FRACFLUX_R', 'FRACFLUX_Z',
                  'FRACMASKED_G', 'FRACMASKED_R', 'FRACMASKED_Z',
                  'FRACIN_G', 'FRACIN_R', 'FRACIN_Z',
                  'NOBS_G', 'NOBS_R', 'NOBS_Z',
                  'PSFDEPTH_G', 'PSFDEPTH_R', 'PSFDEPTH_Z',
                  'GALDEPTH_G', 'GALDEPTH_R', 'GALDEPTH_Z',
                  'FLUX_W1', 'FLUX_W2', 'FLUX_W3', 'FLUX_W4',
                  'FLUX_IVAR_W1', 'FLUX_IVAR_W2', 'FLUX_IVAR_W3', 'FLUX_IVAR_W4',
                  'MW_TRANSMISSION_W1', 'MW_TRANSMISSION_W2', 'MW_TRANSMISSION_W3', 'MW_TRANSMISSION_W4',
                  'ALLMASK_G', 'ALLMASK_R', 'ALLMASK_Z',
                  'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z',
                  'FIBERTOTFLUX_G', 'FIBERTOTFLUX_R', 'FIBERTOTFLUX_Z',
                  'REF_EPOCH',
                  'WISEMASK_W1', 'WISEMASK_W2',
                  'MASKBITS',
                  'LC_FLUX_W1', 'LC_FLUX_W2', 'LC_FLUX_IVAR_W1', 'LC_FLUX_IVAR_W2',
                  'LC_NOBS_W1', 'LC_NOBS_W1', 'LC_MJD_W1', 'LC_MJD_W2',
                  'SHAPE_R', 'SHAPE_E1', 'SHAPE_E2',
                  'SHAPE_R_IVAR', 'SHAPE_E1_IVAR', 'SHAPE_E2_IVAR',
                  'SERSIC', 'SERSIC_IVAR',
                  'REF_ID', 'REF_CAT',
                  'GAIA_PHOT_G_MEAN_MAG', 'GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR',
                  'GAIA_PHOT_BP_MEAN_MAG', 'GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR',
                  'GAIA_PHOT_RP_MEAN_MAG', 'GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR',
                  'GAIA_PHOT_BP_RP_EXCESS_FACTOR', 'GAIA_ASTROMETRIC_EXCESS_NOISE',
                  'GAIA_DUPLICATED_SOURCE', 'GAIA_ASTROMETRIC_SIGMA5D_MAX', 'GAIA_ASTROMETRIC_PARAMS_SOLVED',
                  'PARALLAX', 'PARALLAX_IVAR', 'PMRA', 'PMRA_IVAR', 'PMDEC', 'PMDEC_IVAR'):
            if isinstance(rows[c], MaskedColumn):
                try:
                    assert (rows[c].data.data == rows[c].data.data[0]).all()
                except:
                    try:
                        assert np.isnan(rows[c].data.data).all()
                    except:
                        # print(t, c, rows[c].data.data)
                        if t in anomalous_targetid[key]:
                            anomalous_targetid[key][t].append(c)
                        else:
                            anomalous_targetid[key][t] = [c]
            else:
                try:
                    assert (rows[c] == rows[c][0]).all()
                except:
                    # print(t, rows[c])
                    if t in anomalous_targetid[key]:
                        anomalous_targetid[key][t].append(c)
                    else:
                        anomalous_targetid[key][t] = [c]

In [ ]:
anomalous_targetid['observed']

In [ ]:
anomalous_targetid['potential']

In [ ]:
n_ref_count = 0
for t in anomalous_targetid:
    if len(anomalous_targetid[t]) == 1 and anomalous_targetid[t][0] == 'REF_EPOCH':
        n_ref_count += 1
n_ref_count

In [ ]:
anomaly_counts = np.array([len(anomalous_targetid[t]) for t in anomalous_targetid])

In [ ]:
def inthist(foo, show=False):
    """Create a histogram of integer values.

    Parameters
    ----------
    foo : :class:`numpy.ndarray`
        An array containing integers.
    show : :class:`bool`, optional
        If ``True``, create a histogram and return the matplotlib.axes.Axes
        instance.

    Returns
    -------
    :class:`tuple`
    """
    xmin = min(foo)
    xmax = max(foo)
    x = np.arange( xmin, xmax+1 )
    n = np.zeros( x.shape, dtype=x.dtype )
    for k in range(len(n)):
        n[k] = np.sum( foo == x[k] )
    if show:
        fig = plt.figure(dpi=100)
        ax = fig.add_subplot(111)
        b = ax.bar(x, n, align='center', width=0.5, color='k')
        ax.set_xlim(xmin-1, xmax+1)
        # ax.set_ylim(0, np.ceil(max(n)/10.0)*10.0)
        # ax.set_ylim(0, 10**np.ceil(np.log10(max(n))))
        return (fig, ax)
    else:
        return (n, x)


In [ ]:
fig, ax = inthist(anomaly_counts, show=True)
foo = ax.set_xlabel('Number of columns mismatched')
foo = ax.set_ylabel('Count')
# foo = ax.set_title('targetphot-fuji.fits')
foo = ax.set_title('targetphot-potential-fuji.fits')
# fig.savefig(os.path.join(os.environ['HOME'], 'anomaly_counts.png'))
# fig.savefig(os.path.join(os.environ['HOME'], 'anomaly_counts_potential.png'))

In [ ]:
for t in duplicate_targetid:
    rows = targetphot[duplicate_targetid[t]]
    for c in ('SUBPRIORITY', 'OBSCONDITIONS', 'PRIORITY_INIT', 'NUMOBS_INIT', 'HPXPIXEL',
              'CMX_TARGET', 'DESI_TARGET', 'BGS_TARGET', 'MWS_TARGET',
              'SV1_DESI_TARGET', 'SV1_BGS_TARGET', 'SV1_MWS_TARGET',
              'SV2_DESI_TARGET', 'SV2_BGS_TARGET', 'SV2_MWS_TARGET',
              'SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET',
              'SCND_TARGET', 'SV1_SCND_TARGET', 'SV2_SCND_TARGET', 'SV3_SCND_TARGET',
              'SURVEY', 'PROGRAM', 'TILEID'):
        if isinstance(rows[c], MaskedColumn):
            try:
                assert (rows[c].data.data == rows[c].data.data[0]).all()
            except:
                print(t, c, rows[c].data.data)
        else:
            try:
                assert (rows[c] == rows[c][0]).all()
            except:
                print(t, rows[c])

## Examine ztile files.

In [ ]:
all_targetphotid = list()
ztile_files = glob.glob(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, 'zcatalog', 'ztile-*.fits'))
for f in ztile_files:
    survey = os.path.basename(f).split('-')[1]
    if 'cumulative' in f:
        key = 'LASTNIGHT'
    elif 'perexp' in f:
        key = 'EXPID'
    elif 'pernight' in f:
        key = 'NIGHT'
    else:
        key = 'SPGRPVAL'
    t = Table.read(f, format='fits', hdu='ZCATALOG')
    objid, brickid, release, mock, sky, gaiadr = decode_targetid(t['TARGETID'])
    # print(sky)
    assert (t[key] == t['SPGRPVAL']).all()
    w = (t['TARGETID'] > 0) & (sky == 0)
    all_targetphotid += [targetphotid(x, y, survey) for x, y in zip(t['TARGETID'][w], t['TILEID'][w])]

### Check for uniqueness.

In [ ]:
targetphot_potential_id = [targetphotid(x, y, z) for x, y, z in zip(targetphot_potential['TARGETID'], targetphot_potential['TILEID'], targetphot_potential['SURVEY'])]

In [ ]:
targetphot_id = [targetphotid(x, y, z) for x, y, z in zip(targetphot['TARGETID'], targetphot['TILEID'], targetphot['SURVEY'])]

In [ ]:
for t in targetphot_id:
    assert t in targetphot_potential_id

In [ ]:
for a in all_targetphotid:
    assert a in targetphot_id

### Do ztile files contain extraneous targets?

In [ ]:
ztile = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, 'zcatalog', 'ztile-special-dark-cumulative.fits'), hdu='ZCATALOG')

In [ ]:
w = ztile['TARGETID'] > 0

In [ ]:
np.unique(ztile['TILEID'])

In [ ]:
tid = [targetphotid(x, y, 'special') for x, y in zip(ztile['TARGETID'][w], ztile['TILEID'][w])]

In [ ]:
(158457821078535886096914448642 >> 64) & (2**32 - 1)

In [ ]:
(158457821078535886096914448642 >> 64) >> 32

In [ ]:
(158457821078535886096914448642) & (2**64 - 1)

In [ ]:
ztile[ztile['TARGETID'] == 616088991480938754]

In [ ]:
targetphot[targetphot['TARGETID'] == 616088991480938754]

In [ ]:
targetphot_potential[targetphot_potential['TARGETID'] == 616088991480938754]

In [ ]:
targetphot[targetphot['TILEID'] == 81101]

### Test load targetphot files.

In [ ]:
os.environ['DESI_LOGLEVEL'] = 'DEBUG'
dsr.log = get_logger()
postgresql = dsr.setup_db(schema=specprod+'_target', overwrite=overwrite, hostname='nerscdb03.nersc.gov', verbose=True)

In [ ]:
loader = [{'filepaths': [os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-{specprod}.fits'.format(specprod=os.environ['SPECPROD'])),
                         os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-potential-targets-{specprod}.fits'.format(specprod=os.environ['SPECPROD'])),
                         os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-missing-{specprod}.fits'.format(specprod=os.environ['SPECPROD']))],
           'tcls': dsr.Target,
           'hdu': 'TARGETPHOT',
           'expand': {'DCHISQ': ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_ser',)},
           'q3c': 'ra',
           'chunksize': 100000,
           'maxrows': 0
           },]


In [ ]:
dsr.load_file(**(loader[0]))

## Consistency checks on exposures, frames and tiles

### Load two versions of tiles file

In [ ]:
tiles_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.fits'), hdu='TILE_COMPLETENESS')

In [ ]:
tiles_fits

In [ ]:
tiles_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.csv'), format='ascii.csv')

In [ ]:
tiles_csv

### Are the two tiles files self-consistent?

In [ ]:
for row in range(len(tiles_fits)):
    for col in tiles_fits.colnames:
        try:
            assert tiles_fits[row][col] == tiles_csv[row][col]
        except AssertionError:
            print(tiles_fits[row]['TILEID'], col, tiles_fits[row][col], tiles_csv[row][col])

### Load two versions of exposures file

In [ ]:
exposures_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='EXPOSURES')
frames = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='FRAMES')

In [ ]:
exposures_fits

In [ ]:
exposures_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.csv'), format='ascii.csv')

In [ ]:
exposures_csv

### Are the two exposures files self-consistent?

In [ ]:
for row in range(len(exposures_fits)):
    for col in exposures_fits.colnames:
        try:
            assert exposures_fits[row][col] == exposures_csv[row][col]
        except AssertionError:
            try:
                assert np.around(exposures_fits[row][col].astype(float), 1) == exposures_csv[row][col]
            except AssertionError:
                try:
                    assert np.around(exposures_fits[row][col].astype(float), 2) == exposures_csv[row][col]
                except AssertionError:
                    try:
                        assert np.around(exposures_fits[row][col].astype(float), 3) == exposures_csv[row][col]
                    except AssertionError:
                        print(exposures_fits[row]['TILEID'], col, exposures_fits[row][col], exposures_csv[row][col])

### What programs are present?

In [ ]:
np.unique(faflavor2program(exposures_fits['FAFLAVOR']))

In [ ]:
np.unique(exposures_fits['PROGRAM'])

In [ ]:
np.unique(exposures_fits['GOALTYPE'])

In [ ]:
np.unique(faflavor2program(tiles_fits['FAFLAVOR']))

In [ ]:
np.unique(tiles_fits['PROGRAM'])

In [ ]:
assert (faflavor2program(tiles_fits['FAFLAVOR']) == tiles_fits['PROGRAM']).all

In [ ]:
program = faflavor2program(exposures_fits['FAFLAVOR'])
assert (exposures_fits['PROGRAM'] == program).all()

In [ ]:
for survey in np.unique(exposures_fits['SURVEY']):
    print(f"'{survey}': ", np.unique(program[exposures_fits['SURVEY'] == survey]).tolist(), ',', sep='')

### Compare frames to exposures

In [ ]:
for expid in frames['EXPID']:
    assert expid in exposures_fits['EXPID']

In [ ]:
for k, expid in enumerate(exposures_fits['EXPID']):
    assert (frames['NIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['NIGHT']).all()
    assert (frames['TILEID'][frames['EXPID'] == expid] == exposures_fits[k]['TILEID']).all()
    assert (frames['TILERA'][frames['EXPID'] == expid] == exposures_fits[k]['TILERA']).all()
    assert (frames['TILEDEC'][frames['EXPID'] == expid] == exposures_fits[k]['TILEDEC']).all()
    assert (frames['AIRMASS'][frames['EXPID'] == expid] == exposures_fits[k]['AIRMASS']).all()
    assert (frames['SEEING_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['SEEING_ETC']).all()
    try:
        assert (frames['EFFTIME_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['EFFTIME_ETC']).all()
    except AssertionError:
        print('EFFTIME_ETC', expid, exposures_fits[k]['TILEID'], exposures_fits[k]['EFFTIME_ETC'], frames[column][frames['EFFTIME_ETC'] == expid].tolist())
    # assert (frames['TSNR2_GPBDARK'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBDARK']).all()
    # assert (frames['TSNR2_ELG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_ELG']).all()
    # assert (frames['TSNR2_GPBBRIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBRIGHT']).all()
    # assert (frames['TSNR2_LYA'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LYA']).all()
    # assert (frames['TSNR2_BGS'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_BGS']).all()
    # assert (frames['TSNR2_GPBBACKUP'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBACKUP']).all()
    # assert (frames['TSNR2_QSO'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_QSO']).all()
    # assert (frames['TSNR2_LRG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LRG']).all()
    assert (frames['SURVEY'][frames['EXPID'] == expid] == exposures_fits[k]['SURVEY']).all()
    assert (frames['GOALTYPE'][frames['EXPID'] == expid] == exposures_fits[k]['GOALTYPE']).all()
    assert (frames['FAPRGRM'][frames['EXPID'] == expid] == exposures_fits[k]['FAPRGRM']).all()
    assert (frames['FAFLAVOR'][frames['EXPID'] == expid] == exposures_fits[k]['FAFLAVOR']).all()
    assert (frames['MINTFRAC'][frames['EXPID'] == expid] == exposures_fits[k]['MINTFRAC']).all()
    for column in ('MJD', 'EXPTIME', 'GOALTIME'):
        if column == 'GOALTIME' and (frames[column][frames['EXPID'] == expid] == 0).all():
            print(f"GOALTIME discrepancy for {expid}.")
        else:
            try:
                assert (np.around(frames[column][frames['EXPID'] == expid], 2) == np.around(exposures_fits[k][column], 2)).all()
            except AssertionError:
                try:
                    assert (np.around(frames[column][frames['EXPID'] == expid], 3) == np.around(exposures_fits[k][column], 3)).all()
                except AssertionError:
                    try:
                        assert (np.around(frames[column][frames['EXPID'] == expid], 4) == np.around(exposures_fits[k][column], 4)).all()
                    except AssertionError:
                        pass
                        # print(column, expid, exposures_fits[k][column], frames[column][frames['EXPID'] == expid].tolist())

### Compare tiles to exposures

In [ ]:
for row in tiles_fits:
    w = exposures_fits['TILEID'] == row['TILEID']
    assert len(exposures_fits[w]) == row['NEXP']
    for column in ('SURVEY', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        try:
            assert (exposures_fits[w][column] == row[column]).all()
        except AssertionError:
            print(row['TILEID'], row[column])
            print(exposures_fits[w][column])
    for column in ('TILERA', 'TILEDEC'):
        try:
            assert (np.around(exposures_fits[w][column], 2) == row[column]).all()
        except AssertionError:
            try:
                assert (np.around(exposures_fits[w][column], 3) == row[column]).all()
            except AssertionError:
                assert (np.around(exposures_fits[w][column], 4) == row[column]).all()
    for column in ('EXPTIME', 'EFFTIME_SPEC', 'LRG_EFFTIME_DARK', 'ELG_EFFTIME_DARK', 'BGS_EFFTIME_BRIGHT', 'LYA_EFFTIME_DARK'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
        except AssertionError:
            print(row['TILEID'], row[column])
            print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('EFFTIME_ETC', 'EFFTIME_GFA'):
        if (exposures_fits[w][column] == 0).any():
            assert row[column] == 0
        else:
            try:
                assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('GOALTIME', 'MINTFRAC'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column], 1), row[column])
        except AssertionError:
            try:
                assert np.allclose(np.around(exposures_fits[w][column], 2), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.round(exposures_fits[w][column], decimals=1))
    
    assert exposures_fits[w]['NIGHT'].max() == row['LASTNIGHT']